<center><b>© Content is made available under the CC-BY-NC-ND 4.0 license.

---

Christian Lopez, lopezbec@lafayette.edu<center>

## Este notebook demuestra la generación interactiva de texto utilizando el modelo GPT‑2 Large. Podrás introducir un prompt y luego generar texto de forma interactiva, token por token, o especificar el número de tokens a generar mediante un formulario sencillo. El notebook muestra detalles como la tokenización y las principales predicciones de tokens antes de la generación.

# Acerca de GPT‑2 Large:
## GPT‑2 Large es uno de los modelos de lenguaje de código abierto desarrollado por OpenAI, con 774 millones de parámetros.

In [1]:
# @title 1) Instalando unas librerías de Python y descargando el modelo GPT-2
# Install the transformers library (if not already installed)
!pip install transformers

from IPython.display import clear_output
import torch
import torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Use GPT-2 Large as our model for English text generation
model_name = "gpt2-large"  # GPT-2 Large has ~774M parameters
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()  # Set the model to evaluation mode

# Check if GPU is available and move the model to GPU if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


clear_output()

In [2]:
# @title 2) Aquí puedes escribir tu prompt , para ver como se convierte en una lista de números
# Get a prompt from the user
prompt ='What is the capital of Dominican Repulbic?' # @param {type:"string", placeholder:"Enter a prompt"}

# Tokenize the prompt and print tokens and IDs
tokenized_prompt = tokenizer.tokenize(prompt)
input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

print("\n--- Tokenization Details ---")
print("Tokenized prompt (tokens):", tokenized_prompt)
print("Tokenized prompt (token IDs):", input_ids.tolist()[0])

# Set temperature for randomness (lower value = more deterministic)
temperature = 0.1


--- Tokenization Details ---
Tokenized prompt (tokens): ['What', 'Ġis', 'Ġthe', 'Ġcapital', 'Ġof', 'ĠDominican', 'ĠRep', 'ul', 'b', 'ic', '?']
Tokenized prompt (token IDs): [2061, 318, 262, 3139, 286, 32022, 1432, 377, 65, 291, 30]


In [3]:
# @title 3) Esto es para generar la predicaron de las siguientes palaba, una a la vez



# Compute logits for the current prompt
with torch.no_grad():
    outputs = model(input_ids)
    logits = outputs.logits

# Adjust logits with temperature for randomness and compute softmax probabilities
next_token_logits = logits[:, -1, :] / temperature
probs = F.softmax(next_token_logits, dim=-1)

# Get the top 5 tokens with the highest probabilities
top5 = torch.topk(probs, 5)
top5_ids = top5.indices[0]
top5_probs = top5.values[0]
top5_tokens = [tokenizer.decode(token_id) for token_id in top5_ids]

print("\n--- Top 5 Tokens for the Next Prediction ---")
for token, prob in zip(top5_tokens, top5_probs):
    print(f"Token: '{token.strip()}', Probability: {prob.item():.4f}")

# Initialize generated text with the prompt
generated_text = prompt

# Define the maximum number of tokens to generate in the interactive loop
max_tokens = 50  # Adjust as needed

print("\n--- Interactive Generation Process ---\n")

# Interactive loop: generate tokens one at a time based on user confirmation
for i in range(max_tokens):
    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits

    # Adjust logits with temperature for randomness
    next_token_logits = logits[:, -1, :] / temperature

    # Convert logits to probabilities
    probs = F.softmax(next_token_logits, dim=-1)

    # Sample the next token from the probability distribution
    next_token_id = torch.multinomial(probs, num_samples=1)
    next_token_prob = probs[0, next_token_id.item()]  # probability of the chosen token

    # Decode the predicted token to a string
    next_token = tokenizer.decode(next_token_id.squeeze())

    # Append the predicted token to the input_ids for the next iteration
    input_ids = torch.cat([input_ids, next_token_id], dim=1)
    # Append the predicted token to the generated text
    generated_text += next_token

    # Print the predicted token and its probability
    print(f"Step {i+1}: Predicted token: '{next_token.strip()}' with probability: {next_token_prob.item():.4f}")

    # Ask the user if they want to continue generating the next token
    user_input = input("Generate next token? (y/n): ")
    if user_input.lower() not in ["y", "yes"]:
        break

    # Print the updated prompt/text after the user confirms
    print("\nUpdated text:")
    print(generated_text)
    print("-" * 40)

print("\n--- Full Generated Response ---\n")
print(generated_text)



--- Top 5 Tokens for the Next Prediction ---
Token: '', Probability: 1.0000
Token: 'It', Probability: 0.0000
Token: '', Probability: 0.0000
Token: 'The', Probability: 0.0000
Token: 'Is', Probability: 0.0000

--- Interactive Generation Process ---

Step 1: Predicted token: '' with probability: 1.0000
Generate next token? (y/n): y

Updated text:
What is the capital of Dominican Repulbic?

----------------------------------------
Step 2: Predicted token: '' with probability: 1.0000
Generate next token? (y/n): y

Updated text:
What is the capital of Dominican Repulbic?


----------------------------------------
Step 3: Predicted token: 'The' with probability: 1.0000
Generate next token? (y/n): y

Updated text:
What is the capital of Dominican Repulbic?

The
----------------------------------------
Step 4: Predicted token: 'capital' with probability: 1.0000
Generate next token? (y/n): y

Updated text:
What is the capital of Dominican Repulbic?

The capital
---------------------------------

# 4) Aquí todo de una vez

Trata de cambiar el valor de parametro `temperature` para ser mas "creativo"

In [4]:
# @title

# Get the initial prompt from the user
prompt ='What is the capital of Dominican Repulbic?' # @param {type:"string", placeholder:"Enter a prompt"}

# Tokenize the prompt and encode to tensor
input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
generated_text = prompt

# Ask the user for the number of new tokens to generate
num_tokens = 25 # @param {type:"number"}

# Set temperature for randomness (lower value means more deterministic output)
temperature = 0.1 # @param {type:"slider", min:0.1, max:0.9, step:0.1}

print("\n--- Generating Text ---\n")
for i in range(num_tokens):
    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits

    # Adjust logits with temperature for randomness and compute probabilities
    next_token_logits = logits[:, -1, :] / temperature
    probs = F.softmax(next_token_logits, dim=-1)

    # Sample the next token from the probability distribution
    next_token_id = torch.multinomial(probs, num_samples=1)
    next_token = tokenizer.decode(next_token_id.squeeze())

    # Append the predicted token to input_ids and generated text
    input_ids = torch.cat([input_ids, next_token_id], dim=1)
    generated_text += next_token

    # Optionally, print each step
    print(f"Step {i+1}: Generated token: '{next_token.strip()}'")

print("\n--- Final Generated Text ---\n")
print(generated_text)



--- Generating Text ---

Step 1: Generated token: ''
Step 2: Generated token: ''
Step 3: Generated token: 'The'
Step 4: Generated token: 'capital'
Step 5: Generated token: 'of'
Step 6: Generated token: 'Dominican'
Step 7: Generated token: 'Rep'
Step 8: Generated token: 'ul'
Step 9: Generated token: 'b'
Step 10: Generated token: 'ic'
Step 11: Generated token: 'is'
Step 12: Generated token: 'Sant'
Step 13: Generated token: 'o'
Step 14: Generated token: 'D'
Step 15: Generated token: 'oming'
Step 16: Generated token: 'o'
Step 17: Generated token: '.'
Step 18: Generated token: ''
Step 19: Generated token: ''
Step 20: Generated token: 'What'
Step 21: Generated token: 'is'
Step 22: Generated token: 'the'
Step 23: Generated token: 'capital'
Step 24: Generated token: 'of'
Step 25: Generated token: 'Dominican'

--- Final Generated Text ---

What is the capital of Dominican Repulbic?

The capital of Dominican Repulbic is Santo Domingo.

What is the capital of Dominican


## Recursos Adicionales

Si estás interesado en aprender más sobre los fundamentos de los modelos de lenguaje a gran escala (LLMs) y cómo funcionan, aquí tienes algunos recursos útiles:

### Sitios Web
- **[Documentación de Hugging Face Transformers](https://huggingface.co/transformers/):**  
  Una guía completa sobre cómo utilizar modelos basados en transformadores, que incluye tutoriales y ejemplos de código.
- **[Blog de OpenAI](https://openai.com/blog/):**  
  Lee sobre las últimas investigaciones, aplicaciones y debates acerca de modelos de lenguaje avanzados.
- **[El Transformer Ilustrado](http://jalammar.github.io/illustrated-transformer/):**  
  Una explicación visual e intuitiva de las arquitecturas de transformadores y de cómo impulsan los modelos de lenguaje modernos.
- **[Stanford CS224n: Procesamiento del Lenguaje Natural con Deep Learning](https://web.stanford.edu/class/cs224n/):**  
  Explora apuntes, diapositivas y grabaciones en video sobre PNL y deep learning.

### Canales y Videos en YouTube
- **Hugging Face – Transformers en Acción:**  
  Busca tutoriales prácticos y demostraciones sobre el uso de transformadores en el [Canal de YouTube de Hugging Face](https://www.youtube.com/c/HuggingFace).
- **Yannic Kilcher:**  
  Análisis en profundidad de artículos de investigación y modelos de IA, incluyendo explicaciones de GPT-2, GPT-3 y otros LLMs. Visita su [canal de YouTube](https://www.youtube.com/c/YannicKilcher).
- **Two Minute Papers:**  
  Resúmenes cortos y accesibles de investigaciones recientes en IA, que incluyen avances en modelos de lenguaje. Visita el [Canal de YouTube de Two Minute Papers](https://www.youtube.com/user/keeroyz).
- **DeepLearning.AI:**  
  Contenido educativo y discusiones sobre las tendencias actuales en deep learning e IA. Mira videos en el [Canal de YouTube de DeepLearning.AI](https://www.youtube.com/c/Deeplearningai).

Estos recursos te ayudarán a profundizar en tu comprensión de los LLMs y a explorar más sobre el emocionante campo del procesamiento del lenguaje natural.
